# Finite Element Method for Heat Transfer

## Introduction

The **Finite Element Method (FEM)** is a powerful numerical technique for solving partial differential equations (PDEs) that arise in engineering and physics. In this notebook, we develop a 1D FEM solver for the steady-state heat conduction equation and demonstrate its application to problems with various boundary conditions.

## Governing Equation

The steady-state heat conduction in one dimension is governed by:

$$-\frac{d}{dx}\left(k(x)\frac{dT}{dx}\right) = Q(x)$$

where:
- $T(x)$ is the temperature distribution
- $k(x)$ is the thermal conductivity
- $Q(x)$ is the volumetric heat source

For constant thermal conductivity $k$, this simplifies to:

$$-k\frac{d^2T}{dx^2} = Q(x)$$

## Weak Formulation

The FEM is based on the **weak (variational) formulation** of the PDE. Multiplying by a test function $v(x)$ and integrating over the domain $\Omega = [0, L]$:

$$\int_0^L -k\frac{d^2T}{dx^2} v \, dx = \int_0^L Q(x) v \, dx$$

Applying integration by parts:

$$\int_0^L k\frac{dT}{dx}\frac{dv}{dx} \, dx - \left[k\frac{dT}{dx}v\right]_0^L = \int_0^L Q(x) v \, dx$$

With appropriate boundary conditions (Dirichlet: $T$ prescribed, or Neumann: $q = -k\frac{dT}{dx}$ prescribed), this becomes the weak form we solve.

## Galerkin Discretization

We approximate the temperature field using piecewise linear **shape functions** $N_i(x)$:

$$T(x) \approx T^h(x) = \sum_{i=1}^{n} T_i N_i(x)$$

where $T_i$ are the nodal temperature values and $n$ is the number of nodes.

### Linear Shape Functions

For element $e$ spanning from $x_i$ to $x_{i+1}$ with length $h_e = x_{i+1} - x_i$, the local shape functions are:

$$N_1^{(e)}(\xi) = \frac{1 - \xi}{2}, \quad N_2^{(e)}(\xi) = \frac{1 + \xi}{2}$$

where $\xi \in [-1, 1]$ is the local coordinate related to $x$ by:

$$x = \frac{x_i + x_{i+1}}{2} + \frac{h_e}{2}\xi$$

## Element Stiffness Matrix

The element stiffness matrix for a linear element is:

$$\mathbf{K}^{(e)} = \frac{k}{h_e}\begin{bmatrix} 1 & -1 \\ -1 & 1 \end{bmatrix}$$

## Element Load Vector

For a uniform heat source $Q$ over the element:

$$\mathbf{f}^{(e)} = \frac{Q h_e}{2}\begin{bmatrix} 1 \\ 1 \end{bmatrix}$$

## Global System Assembly

The global stiffness matrix $\mathbf{K}$ and load vector $\mathbf{f}$ are assembled from element contributions. The resulting linear system is:

$$\mathbf{K}\mathbf{T} = \mathbf{f}$$

After applying boundary conditions, we solve for the nodal temperatures $\mathbf{T}$.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import lil_matrix
from scipy.sparse.linalg import spsolve

# Set up publication-quality plots
plt.rcParams.update({
    'font.size': 12,
    'axes.labelsize': 14,
    'axes.titlesize': 14,
    'legend.fontsize': 11,
    'figure.figsize': (10, 8),
    'lines.linewidth': 2
})

## Implementation of 1D FEM Heat Transfer Solver

We now implement a modular FEM solver for 1D steady-state heat conduction.

In [ ]:
class FEM1DHeatSolver:
    """
    1D Finite Element Method solver for steady-state heat conduction.
    
    Solves: -k * d²T/dx² = Q(x) on [0, L]
    with Dirichlet and/or Neumann boundary conditions.
    """
    
    def __init__(self, L, n_elements, k=1.0):
        """
        Initialize the FEM solver.
        
        Parameters:
        -----------
        L : float
            Domain length
        n_elements : int
            Number of finite elements
        k : float or array
            Thermal conductivity (constant or per-element)
        """
        self.L = L
        self.n_elements = n_elements
        self.n_nodes = n_elements + 1
        
        # Generate uniform mesh
        self.nodes = np.linspace(0, L, self.n_nodes)
        self.h = L / n_elements  # Element size (uniform)
        
        # Thermal conductivity
        if np.isscalar(k):
            self.k = np.full(n_elements, k)
        else:
            self.k = np.array(k)
        
        # Initialize global matrices
        self.K = None
        self.f = None
        self.T = None
    
    def element_stiffness(self, elem_idx):
        """
        Compute element stiffness matrix for linear elements.
        
        K_e = (k/h) * [[1, -1], [-1, 1]]
        """
        k_e = self.k[elem_idx]
        h_e = self.h
        return (k_e / h_e) * np.array([[1, -1], [-1, 1]])
    
    def element_load(self, elem_idx, Q_func):
        """
        Compute element load vector using 2-point Gaussian quadrature.
        
        f_e = integral(Q(x) * N_i dx)
        """
        # Gauss points and weights for 2-point quadrature on [-1, 1]
        xi_gauss = np.array([-1/np.sqrt(3), 1/np.sqrt(3)])
        w_gauss = np.array([1.0, 1.0])
        
        # Element node positions
        x1 = self.nodes[elem_idx]
        x2 = self.nodes[elem_idx + 1]
        h_e = self.h
        
        f_e = np.zeros(2)
        
        for xi, w in zip(xi_gauss, w_gauss):
            # Map to physical coordinates
            x = 0.5 * (x1 + x2) + 0.5 * h_e * xi
            
            # Shape functions at Gauss point
            N = np.array([0.5 * (1 - xi), 0.5 * (1 + xi)])
            
            # Jacobian (for linear elements: h/2)
            J = h_e / 2
            
            # Add contribution
            f_e += Q_func(x) * N * J * w
        
        return f_e
    
    def assemble(self, Q_func=lambda x: 0):
        """
        Assemble global stiffness matrix and load vector.
        """
        n = self.n_nodes
        self.K = lil_matrix((n, n))
        self.f = np.zeros(n)
        
        for e in range(self.n_elements):
            # Element stiffness and load
            K_e = self.element_stiffness(e)
            f_e = self.element_load(e, Q_func)
            
            # Global node indices for this element
            nodes_e = [e, e + 1]
            
            # Assembly
            for i_local, i_global in enumerate(nodes_e):
                self.f[i_global] += f_e[i_local]
                for j_local, j_global in enumerate(nodes_e):
                    self.K[i_global, j_global] += K_e[i_local, j_local]
        
        self.K = self.K.tocsr()
    
    def apply_dirichlet_bc(self, node_idx, value):
        """
        Apply Dirichlet (essential) boundary condition: T(node) = value
        Uses the penalty method for simplicity.
        """
        penalty = 1e20
        self.K = self.K.tolil()
        self.K[node_idx, node_idx] += penalty
        self.f[node_idx] += penalty * value
        self.K = self.K.tocsr()
    
    def apply_neumann_bc(self, node_idx, flux):
        """
        Apply Neumann (natural) boundary condition: q = -k*dT/dx = flux
        This adds the flux contribution to the load vector.
        """
        # Note: positive flux means heat entering the domain
        if node_idx == 0:
            self.f[node_idx] -= flux  # Left boundary
        else:
            self.f[node_idx] += flux  # Right boundary
    
    def solve(self):
        """
        Solve the linear system K*T = f.
        """
        self.T = spsolve(self.K, self.f)
        return self.T
    
    def get_heat_flux(self):
        """
        Compute heat flux at element centers: q = -k * dT/dx
        """
        flux = np.zeros(self.n_elements)
        x_centers = np.zeros(self.n_elements)
        
        for e in range(self.n_elements):
            dT_dx = (self.T[e + 1] - self.T[e]) / self.h
            flux[e] = -self.k[e] * dT_dx
            x_centers[e] = 0.5 * (self.nodes[e] + self.nodes[e + 1])
        
        return x_centers, flux

## Test Case 1: Uniform Heat Source with Dirichlet Boundaries

Consider a rod of length $L = 1$ m with:
- Thermal conductivity $k = 1$ W/(m·K)
- Uniform heat source $Q = 10$ W/m³
- Boundary conditions: $T(0) = 0$, $T(L) = 0$

### Analytical Solution

The analytical solution for this problem is:

$$T(x) = \frac{Q}{2k}x(L - x)$$

This is a parabola with maximum at $x = L/2$.

In [ ]:
# Problem parameters
L = 1.0  # Domain length (m)
k = 1.0  # Thermal conductivity (W/m·K)
Q_val = 10.0  # Heat source (W/m³)
T_left = 0.0  # Left boundary temperature (°C)
T_right = 0.0  # Right boundary temperature (°C)

# Analytical solution
def analytical_solution_case1(x):
    return (Q_val / (2 * k)) * x * (L - x)

# Compare FEM solutions with different mesh densities
n_elements_list = [4, 8, 16, 32]
colors = plt.cm.viridis(np.linspace(0, 0.8, len(n_elements_list)))

fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# Plot 1: Temperature distribution comparison
ax1 = axes[0, 0]
x_analytical = np.linspace(0, L, 200)
ax1.plot(x_analytical, analytical_solution_case1(x_analytical), 'k-', 
         linewidth=2.5, label='Analytical')

errors = []
for n_elem, color in zip(n_elements_list, colors):
    solver = FEM1DHeatSolver(L, n_elem, k)
    solver.assemble(Q_func=lambda x: Q_val)
    solver.apply_dirichlet_bc(0, T_left)
    solver.apply_dirichlet_bc(n_elem, T_right)
    T = solver.solve()
    
    ax1.plot(solver.nodes, T, 'o-', color=color, 
             markersize=8, label=f'FEM ({n_elem} elements)')
    
    # Compute L2 error
    T_exact = analytical_solution_case1(solver.nodes)
    error = np.sqrt(np.mean((T - T_exact)**2))
    errors.append(error)

ax1.set_xlabel('Position x (m)')
ax1.set_ylabel('Temperature T (°C)')
ax1.set_title('Case 1: Uniform Heat Source with Dirichlet BC')
ax1.legend(loc='upper right')
ax1.grid(True, alpha=0.3)

# Plot 2: Convergence study
ax2 = axes[0, 1]
h_values = [L / n for n in n_elements_list]
ax2.loglog(h_values, errors, 'bo-', markersize=10, linewidth=2, label='FEM Error')

# Theoretical O(h²) convergence line
h_ref = np.array(h_values)
error_ref = errors[0] * (h_ref / h_ref[0])**2
ax2.loglog(h_ref, error_ref, 'r--', linewidth=2, label=r'$O(h^2)$ reference')

ax2.set_xlabel('Element size h (m)')
ax2.set_ylabel('L² Error')
ax2.set_title('Convergence Study')
ax2.legend()
ax2.grid(True, alpha=0.3, which='both')

print("Convergence rates:")
for i in range(1, len(errors)):
    rate = np.log(errors[i-1]/errors[i]) / np.log(2)
    print(f"  {n_elements_list[i-1]} -> {n_elements_list[i]} elements: rate = {rate:.3f}")

## Test Case 2: Mixed Boundary Conditions

Now consider:
- Left boundary: Dirichlet $T(0) = 100°C$
- Right boundary: Neumann (insulated) $q(L) = 0$
- No internal heat source: $Q = 0$

The analytical solution is simply $T(x) = 100$ (constant temperature).

### Case 2b: Heat flux at right boundary

With $T(0) = 100°C$ and $q(L) = -50$ W/m² (heat leaving), the solution is:

$$T(x) = 100 - \frac{q}{k}x = 100 + 50x$$

In [ ]:
# Case 2b: Dirichlet at left, heat flux at right
n_elem = 20
T_left = 100.0
q_right = -50.0  # Heat flux leaving domain (negative = outward)

solver2 = FEM1DHeatSolver(L, n_elem, k)
solver2.assemble(Q_func=lambda x: 0)
solver2.apply_dirichlet_bc(0, T_left)
solver2.apply_neumann_bc(n_elem, q_right)
T2 = solver2.solve()

# Analytical solution
def analytical_case2(x):
    return T_left - (q_right / k) * x

# Plot
ax3 = axes[1, 0]
x_analytical = np.linspace(0, L, 200)
ax3.plot(x_analytical, analytical_case2(x_analytical), 'k-', 
         linewidth=2.5, label='Analytical')
ax3.plot(solver2.nodes, T2, 'ro-', markersize=6, label='FEM')

ax3.set_xlabel('Position x (m)')
ax3.set_ylabel('Temperature T (°C)')
ax3.set_title('Case 2: Mixed BC (Dirichlet + Neumann)')
ax3.legend()
ax3.grid(True, alpha=0.3)

# Compute and display heat flux
x_flux, flux = solver2.get_heat_flux()
print(f"Heat flux (should be constant at {-q_right:.1f} W/m²):")
print(f"  Mean flux: {np.mean(flux):.4f} W/m²")
print(f"  Flux std: {np.std(flux):.2e} W/m²")

## Test Case 3: Variable Thermal Conductivity

Consider a composite rod with two materials:
- Material 1 ($0 \leq x < 0.5$): $k_1 = 1$ W/(m·K)
- Material 2 ($0.5 \leq x \leq 1$): $k_2 = 5$ W/(m·K)

With $T(0) = 0$ and $T(L) = 100°C$, no heat source.

### Analytical Solution

The heat flux must be continuous: $q = -k_1 \frac{dT_1}{dx} = -k_2 \frac{dT_2}{dx}$

The temperature profile consists of two linear segments with slopes inversely proportional to conductivity.

In [ ]:
# Case 3: Variable conductivity (composite rod)
n_elem = 40
k1, k2 = 1.0, 5.0
T_left, T_right = 0.0, 100.0
x_interface = 0.5

# Create conductivity array
solver3 = FEM1DHeatSolver(L, n_elem, k=1.0)  # Initialize with dummy k

# Assign conductivity based on element center position
k_array = np.zeros(n_elem)
for e in range(n_elem):
    x_center = (solver3.nodes[e] + solver3.nodes[e + 1]) / 2
    k_array[e] = k1 if x_center < x_interface else k2
solver3.k = k_array

solver3.assemble(Q_func=lambda x: 0)
solver3.apply_dirichlet_bc(0, T_left)
solver3.apply_dirichlet_bc(n_elem, T_right)
T3 = solver3.solve()

# Analytical solution for composite rod
# Heat flux q is constant: q = (T_right - T_left) / (L1/k1 + L2/k2)
L1 = x_interface
L2 = L - x_interface
R_total = L1/k1 + L2/k2  # Total thermal resistance
q_composite = (T_right - T_left) / R_total
T_interface = T_left + q_composite * L1 / k1

def analytical_case3(x):
    if np.isscalar(x):
        if x < x_interface:
            return T_left + (q_composite / k1) * x
        else:
            return T_interface + (q_composite / k2) * (x - x_interface)
    else:
        result = np.zeros_like(x)
        mask1 = x < x_interface
        mask2 = ~mask1
        result[mask1] = T_left + (q_composite / k1) * x[mask1]
        result[mask2] = T_interface + (q_composite / k2) * (x[mask2] - x_interface)
        return result

# Plot
ax4 = axes[1, 1]
x_analytical = np.linspace(0, L, 200)
ax4.plot(x_analytical, analytical_case3(x_analytical), 'k-', 
         linewidth=2.5, label='Analytical')
ax4.plot(solver3.nodes, T3, 'go-', markersize=4, label='FEM')

# Mark interface
ax4.axvline(x=x_interface, color='gray', linestyle='--', alpha=0.7)
ax4.text(0.25, 80, f'$k_1 = {k1}$', fontsize=12, ha='center')
ax4.text(0.75, 80, f'$k_2 = {k2}$', fontsize=12, ha='center')

ax4.set_xlabel('Position x (m)')
ax4.set_ylabel('Temperature T (°C)')
ax4.set_title('Case 3: Composite Rod (Variable k)')
ax4.legend()
ax4.grid(True, alpha=0.3)

print(f"Interface temperature:")
print(f"  Analytical: {T_interface:.4f} °C")
interface_idx = int(n_elem * x_interface / L)
print(f"  FEM: {T3[interface_idx]:.4f} °C")
print(f"  Heat flux: {q_composite:.4f} W/m²")

plt.tight_layout()
plt.savefig('plot.png', dpi=150, bbox_inches='tight')
plt.show()

print("\nFigure saved to 'plot.png'")

## Summary and Key Takeaways

### What We Demonstrated

1. **FEM Fundamentals**: The finite element method transforms a PDE into a system of linear equations through:
   - Weak formulation (integration by parts)
   - Discretization with shape functions
   - Assembly of element contributions

2. **Convergence**: Linear elements exhibit $O(h^2)$ convergence in the L² norm, meaning halving the element size reduces error by a factor of 4.

3. **Boundary Conditions**:
   - **Dirichlet (essential)**: Applied by modifying the stiffness matrix
   - **Neumann (natural)**: Automatically satisfied or added to load vector

4. **Variable Properties**: FEM naturally handles spatially varying material properties by using element-wise conductivity values.

### Extensions

This 1D implementation can be extended to:
- **2D/3D problems** using triangular/tetrahedral or quadrilateral/hexahedral elements
- **Transient heat transfer** by adding the heat capacity matrix
- **Nonlinear problems** where $k = k(T)$ using Newton-Raphson iteration
- **Convective boundary conditions**: $q = h(T - T_\infty)$

### Governing Equations Summary

| Quantity | Symbol | Equation |
|----------|--------|----------|
| Heat conduction | Fourier's Law | $q = -k\nabla T$ |
| Steady-state | Heat equation | $-\nabla \cdot (k\nabla T) = Q$ |
| Element stiffness | Linear 1D | $K^e = \frac{k}{h}\begin{bmatrix}1 & -1\\-1 & 1\end{bmatrix}$ |
| Thermal resistance | Series | $R = \sum_i \frac{L_i}{k_i A}$ |